# Experimentation on WAn VACE. Inference Otimization

there are several experiments, that incremets the complexity
1. One process that uses only 4 GPUS H100. Only one task of video generatio. Total time 262.15 sec
2. Two processes. Each process uses 4 GPUS H100 (so the 8XH100 GPUS are used). Each process with only one task. Total time 263.31 sec
3. Two processes. Each process uses 4 GPUS H100 (so the 8XH100 GPUS are used). Each process with 10 tasks. Total time 1850 sec


# Experiment 1
One process that uses only 4 GPUS H100. Only one task of video generatio. Total time 262.15 sec

In [3]:
import time

start_time = time.time()

!torchrun --nproc_per_node=4 \
  /workspace/Wan2.1/generate.py \
  --task vace-14B \
  --size 832*480 \
  --ckpt_dir /workspace/Wan2.1-VACE-14B \
  --dit_fsdp \
  --t5_fsdp \
  --ulysses_size 4 \
  --src_ref_images /workspace/experiments/experiments_inpaiting_outpainting_mask/data/licuadora1.png \
  --prompt "A wide modern kitchen environment featuring a black Oster blender filled with fruits and ice on a white marble counter. Extend the current countertop seamlessly to the left and right, adding clean kitchen utensils, fresh fruit bowls, and a cutting board. Behind the blender, a tiled backsplash and white cabinets continue across the scene. Include soft natural lighting and subtle reflections on the countertop for a realistic look. Maintain the color tones and material style of the original image." \
  --save_file /workspace/experiments/inference_optimzation/output/output_licuadora_v1_832_480.mp4

end_time = time.time()
print(f"⏱️ Tiempo de ejecución: {end_time - start_time:.2f} segundos")


W0724 17:08:26.385000 140517213615936 torch/distributed/run.py:779] 
W0724 17:08:26.385000 140517213615936 torch/distributed/run.py:779] *****************************************
W0724 17:08:26.385000 140517213615936 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0724 17:08:26.385000 140517213615936 torch/distributed/run.py:779] *****************************************
[2025-07-24 17:08:32,821] INFO: offload_model is not specified, set to False.
DEBUG 07-24 17:08:33 [parallel_state.py:207] world_size=4 rank=0 local_rank=-1 distributed_init_method=env:// backend=nccl
DEBUG 07-24 17:08:33 [parallel_state.py:207] world_size=4 rank=1 local_rank=-1 distributed_init_method=env:// backend=nccl
DEBUG 07-24 17:08:33 [parallel_state.py:207] world_size=4 rank=2 local_rank=-1 distributed_init_method=env://

# Experiment 2

Two processes. Each process uses 4 GPUS H100 (so the 8XH100 GPUS are used). Each process with only one task. Total time 263.31 sec


In [5]:
import time
import subprocess
import os
from pathlib import Path

# === CONFIGURACIÓN GENERAL ===
output_dir = Path("/workspace/experiments/inference_optimzation/output")
log_dir = output_dir / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

start_time = time.time()

# === COMANDO 1 - GPUs 0-3, puerto 29501 ===
log1_path = log_dir / "v1.log"
log1 = open(log1_path, "w")

proc1 = subprocess.Popen([
    "torchrun", "--nproc_per_node=4", "--master_port=29501",
    "/workspace/Wan2.1/generate.py",
    "--task", "vace-14B",
    "--size", "832*480",
    "--ckpt_dir", "/workspace/Wan2.1-VACE-14B",
    "--dit_fsdp",
    "--t5_fsdp",
    "--ulysses_size", "4",
    "--src_ref_images", str(output_dir / "licuadora1.png"),
    "--prompt", "A wide modern kitchen environment featuring a black Oster blender filled with fruits and ice on a white marble counter. Extend the current countertop seamlessly to the left and right, adding clean kitchen utensils, fresh fruit bowls, and a cutting board. Behind the blender, a tiled backsplash and white cabinets continue across the scene. Include soft natural lighting and subtle reflections on the countertop for a realistic look. Maintain the color tones and material style of the original image.",
    "--save_file", str(output_dir / "output_licuadora_v1_832_480.mp4")
], env={**os.environ, "CUDA_VISIBLE_DEVICES": "0,1,2,3"}, stdout=log1, stderr=log1)

# === COMANDO 2 - GPUs 4-7, puerto 29502 ===
log2_path = log_dir / "v2.log"
log2 = open(log2_path, "w")

proc2 = subprocess.Popen([
    "torchrun", "--nproc_per_node=4", "--master_port=29502",
    "/workspace/Wan2.1/generate.py",
    "--task", "vace-14B",
    "--size", "832*480",
    "--ckpt_dir", "/workspace/Wan2.1-VACE-14B",
    "--dit_fsdp",
    "--t5_fsdp",
    "--ulysses_size", "4",
    "--src_ref_images", str(output_dir / "licuadora1.png"),
    "--prompt", "A wide modern kitchen environment featuring a black Oster blender filled with fruits and ice on a white marble counter. Extend the current countertop seamlessly to the left and right, adding clean kitchen utensils, fresh fruit bowls, and a cutting board. Behind the blender, a tiled backsplash and white cabinets continue across the scene. Include soft natural lighting and subtle reflections on the countertop for a realistic look. Maintain the color tones and material style of the original image.",
    "--save_file", str(output_dir / "output_licuadora_v2_832_480.mp4")
], env={**os.environ, "CUDA_VISIBLE_DEVICES": "4,5,6,7"}, stdout=log2, stderr=log2)

# === ESPERAR A QUE AMBOS TERMINEN ===
proc1.wait()
proc2.wait()

end_time = time.time()
print(f"⏱️ Tiempo total de ejecución: {end_time - start_time:.2f} segundos")

# === CERRAR LOGS ===
log1.close()
log2.close()


⏱️ Tiempo total de ejecución: 263.31 segundos


# Experiment 3
Two processes. Each process uses 4 GPUS H100 (so the 8XH100 GPUS are used). Each process with 10 tasks. Total time 1850 sec


In [2]:
import time
import subprocess
import os
from pathlib import Path

# === CONFIGURACIÓN DE DIRECTORIOS ===
output_dir = Path("/workspace/inference_optimization/output")
log_dir = output_dir / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# === RUTAS DE LOG ===
log1_path = log_dir / "generate_vace_1_gpus_0_3.txt"
log2_path = log_dir / "generate_vace_1_gpus_4_7.txt"
time_log_path = log_dir / "tiempo_total.txt"

# === TIEMPO DE INICIO GENERAL ===
global_start = time.time()

# === COMANDO 1: GPUs 0–3 ===
start1 = time.time()
cmd1 = f"""
CUDA_VISIBLE_DEVICES=0,1,2,3 torchrun --nproc_per_node=4 --master_port=29501 /workspace/Wan2.1/generate_vace_1.py 2>&1 | tee {log1_path}
"""
proc1 = subprocess.Popen(["bash", "-c", cmd1])

# === COMANDO 2: GPUs 4–7 ===
start2 = time.time()
cmd2 = f"""
CUDA_VISIBLE_DEVICES=4,5,6,7 torchrun --nproc_per_node=4 --master_port=29502 /workspace/Wan2.1/generate_vace_2.py 2>&1 | tee {log2_path}
"""
proc2 = subprocess.Popen(["bash", "-c", cmd2])

# === ESPERAR A QUE TERMINEN ===
proc1.wait()
end1 = time.time()

proc2.wait()
end2 = time.time()

# === DURACIONES ===
duration1 = end1 - start1
duration2 = end2 - start2
total_duration = time.time() - global_start

# === AGREGAR TIEMPO A LOS LOGS ===
with open(log1_path, "a") as f1:
    f1.write(f"\n🕒 Tiempo de ejecución proceso 1 (GPUs 0–3): {duration1:.2f} segundos\n")
with open(log2_path, "a") as f2:
    f2.write(f"\n🕒 Tiempo de ejecución proceso 2 (GPUs 4–7): {duration2:.2f} segundos\n")

# === LOG DE TIEMPO GLOBAL ===
with open(time_log_path, "a") as time_log:
    time_log.write(
        f"🕒 Tiempo proceso 1: {duration1:.2f} s | "
        f"🕒 Tiempo proceso 2: {duration2:.2f} s | "
        f"🕒 Tiempo total (ambos procesos): {total_duration:.2f} s\n"
    )

# === CONFIRMACIÓN FINAL ===
print("✅ Procesos finalizados.")
print(f"📄 Log proceso 1: {log1_path}")
print(f"📄 Log proceso 2: {log2_path}")
print(f"📄 Tiempo total: {time_log_path}")


W0725 15:40:25.042000 140105320523584 torch/distributed/run.py:779] 
W0725 15:40:25.042000 140105320523584 torch/distributed/run.py:779] *****************************************
W0725 15:40:25.042000 140105320523584 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0725 15:40:25.042000 140105320523584 torch/distributed/run.py:779] *****************************************
W0725 15:40:25.042000 139796101019456 torch/distributed/run.py:779] 
W0725 15:40:25.042000 139796101019456 torch/distributed/run.py:779] *****************************************
W0725 15:40:25.042000 139796101019456 torch/distributed/run.py:779] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your 